In [3]:
# Question - 1
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
link = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
path = tf.keras.utils.get_file('mnist.npz' , link)
data = np.load(path)
x_coord_train , y_coord_train = data["x_train"], data["y_train"]
x_coord_test , y_coord_test = data["x_test"], data["y_test"]
#print(x_coord_train.shape)
#print(y_coord_train.shape)
x_coord_train_vectorized = x_coord_train.reshape(x_coord_train.shape[0],-1)
x_coord_test_vectorized = x_coord_test.reshape(x_coord_test.shape[0],-1)

In [ ]:
samples = {val:[] for val in range(10)}

for i in range(len(y_coord_train)):
    val = y_coord_train[i]
    if len(samples[val]) < 5:
        samples[val].append(x_coord_train[i])


plt.figure(figsize=(5,10))


for i in range(10):
    for j in range(5):
        plt.subplot(10, 5, i*5 + j + 1)
        plt.imshow(samples[i][j], cmap='gray')
        plt.axis("off")
plt.show()

# vectorizing the images
#print(x_coord_train.shape)
#print(x_coord_test.shape)
#print(x_coord_train_vectorized.shape)
#print(x_coord_test_vectorized.shape)
#computing the mean and covariance of the classes
mean_of_classes = []
covariance_of_classes = []
for i in range(10):
    curr_class = []
    for j in range(len(x_coord_train_vectorized)):
        if y_coord_train[j] == i:
            curr_class.append(x_coord_train_vectorized[j])
    curr_mean = np.mean(curr_class , axis= 0)
    mean_of_classes.append(curr_mean)
    curr_covariance = np.cov(curr_class ,rowvar= False)
    covariance_of_classes.append(curr_covariance)
#print(mean_of_classes[i].shape)
# computing the prior probabilities
dict_count = {}
for i in range(10) :
    for j in range(len(x_coord_train_vectorized)) :
        if y_coord_train[j] == i and i not in dict_count :
            dict_count[i] = 1
        elif y_coord_train[j] == i and i in dict_count :
            dict_count[i] += 1
#computing the QDA on the given dataset using the formula
inverse_cov = []
for i in range(10) :
    #print(np.linalg.det(covariance_of_classes[i] + (0.000001)*(np.identity(covariance_of_classes[i].shape[0]))))
    inverse_cov.append(np.linalg.inv(covariance_of_classes[i] + (1e-6)*(np.identity(covariance_of_classes[i].shape[0]))))
def QDA1(mean , inverse_cov , x , prior):
    # print(x.shape, inverse_cov.shape, mean.shape)
    return (np.transpose(x) @ (-0.5 * inverse_cov) @ x) + (np.transpose(inverse_cov @ mean ) @ x) + (-0.5 * np.transpose(mean) @ inverse_cov @ mean) + prior
#print("The accuracies are :\n")
matching_count = 0
class_count_fromQDA = {}
class_count_inreal = {}
for i in range(len(x_coord_test_vectorized)):
    if y_coord_test[i] not in class_count_inreal :
        class_count_inreal[y_coord_test[i]] = 1
    elif y_coord_test[i] in class_count_inreal :
         class_count_inreal[y_coord_test[i]] += 1
for i in range(len(x_coord_test_vectorized)):
    discriminant_values = []
    for j in range(10) :
        discriminant_values.append(QDA1(mean_of_classes[j] , inverse_cov[j] , x_coord_test_vectorized[i] , np.log(dict_count[j]/60000)))
    #print(discriminant_values)
    max_index = np.argmax(discriminant_values)
    #print(max_index)
    if max_index == y_coord_test[i]:
        matching_count +=1
        if max_index not in class_count_fromQDA:
            class_count_fromQDA[max_index] = 1
        elif max_index in class_count_fromQDA :
            class_count_fromQDA[max_index] += 1
print("Accuracy is: ",matching_count / 100,"%")

for x in class_count_fromQDA :
    print("Accuracy of class ", x ," is: ",(class_count_fromQDA[x]/class_count_inreal[x])*100 ,"%")

In [8]:
#Question 2

# taking the 100 samples from each class and creating 784 X 1000 data matrix .
new_train_set = []

for i in range(10):
    class_samples = []
    counter = 0
    for j in range(len(x_coord_train_vectorized)):
        if y_coord_train[j] == i:
            class_samples.append(x_coord_train_vectorized[j])
            counter += 1
            if counter == 100:
                break
    new_train_set.extend(class_samples)
X = np.array(new_train_set)
print(X.shape)
X = X.T
print(X.shape)

(1000, 784)
(784, 1000)


In [6]:
# removing the mean from X .
mean_of_X = np.mean(X,axis=1,keepdims=True)
print(mean_of_X.shape)
X_centralized = X-mean_of_X
#print(X_centralized.shape)
# computing covariance S of X
S = (X_centralized @ X_centralized.T )/ 999

S_eigenvalues , S_eigenvectors = np.linalg.eigh(S)
#print(S_eigenvectors)
sorted_S  = np.argsort(S_eigenvalues)[::-1]
S_eigenvalues = S_eigenvalues[sorted_S]
S_eigenvectors = S_eigenvectors[:,sorted_S]
#print(S_eigenvalues)
U = S_eigenvectors
#print(U.shape)
Y = U.T @ X_centralized
X_recon = U @ Y  + mean_of_X
#print(X_recon.shape)
MSE = 0

for i in range(len(X)):
    for j in range(len(X[i])):
        MSE += ((X[i][j] - X_recon[i][j])**2)

print("The MSE of coming out to be:",MSE)
print("----------------------------------")
# finding QDA for Xtest for all values of p.
p_values = [ 5 ,10 ,20]
for p in p_values :
    print("Image for p=",p,"is:")
    Up = U[:,:p]
    Y = Up.T @ X_centralized
    X_recon = (Up @ Y) + mean_of_X
    reconstruction_of_image = X_recon.reshape(28,28,-1)
    plt.figure(figsize=(5,10))
    for i in range(10):
        for j in range(5) :
            plt.subplot(10,5,i*5 + j +1)
            plt.imshow(reconstruction_of_image[:,:,i*100+j],cmap='gray')
            plt.axis('off')
    plt.show()
    print("------------------------------------------------------")
new_train_set = []
prior_prob = {}
for i in range(10):
    class_samples = []
    counter = 0
    for j in range(len(x_coord_train_vectorized)):
        if y_coord_train[j] == i:
            class_samples.append(x_coord_train_vectorized[j])
            counter += 1
            if counter == 6000: # approx taking the whole dataset , as for some categories number of values may exceed 6000 , but we take them as 6000 and break .
                break
    prior_prob[i] = counter
    new_train_set.extend(class_samples)
X = np.array(new_train_set)
X = X.T
print(X.shape)
mean_of_X = np.mean(X,axis=1,keepdims=True)
X_centralized = X-mean_of_X
S = (X_centralized @ X_centralized.T )/ 59999

S_eigenvalues , S_eigenvectors = np.linalg.eigh(S)
#print(S_eigenvectors)
sorted_S  = np.argsort(S_eigenvalues)[::-1]
S_eigenvalues = S_eigenvalues[sorted_S]
S_eigenvectors = S_eigenvectors[:,sorted_S]
#print(S_eigenvalues)
U = S_eigenvectors
mean_of_X = mean_of_X.T
X_centralized = X_centralized.T
#print(X.shape , mean_of_X.shape , X_centralized.shape)
# for x in prior_prob :
#   print(prior_prob[x])
for p in p_values :
    Up = U[:,:p]
  #  print(X_centralized.shape , Up.shape)
    Y = np.dot(X_centralized,Up)
    new_mean = []
    new_cov = []
    new_inverse_cov =[]
    for i in range(10) :
        sample_set = Y[i*6000 : (i+1)*6000]
       #print(sample_set)
        new_mean.append(np.mean(sample_set  ,axis= 0))
        new_cov.append(np.cov(sample_set , rowvar=False))
    for i in range(10) :
        new_inverse_cov.append(np.linalg.inv(new_cov[i] + (1e-6)*np.identity(new_cov[i].shape[0])))
    #print(x_coord_test_vectorized.shape, mean_of_X.T.shape)
    Y_test = np.dot(x_coord_test_vectorized - mean_of_X , Up)
    matching_count_short = 0
    class_short_in_real = {}
    class_short_in_QDA = {}
    for i in range(len(Y_test)) :
        if y_coord_test[i] not in class_short_in_real :
            class_short_in_real[y_coord_test[i]] = 1
        elif y_coord_test[i] in class_short_in_real:
            class_short_in_real[y_coord_test[i]] += 1

    for i in range(len(Y_test)) :
        discriminant_values_short = []
        for j in range(10) :
            discriminant_values_short.append(QDA1(new_mean[j] , new_inverse_cov[j] , Y_test[i] ,np.log(prior_prob[j]) ))
        max_index_short = np.argmax(discriminant_values_short)
        if max_index_short == y_coord_test[i] :
            matching_count_short += 1
            if max_index_short not in class_short_in_QDA :
                class_short_in_QDA[max_index_short] = 1
            elif max_index_short in class_short_in_QDA :
                class_short_in_QDA[max_index_short]+=1
    print("Accuracy for p=",p,"is:",matching_count_short / 100,"%")

    for x in class_short_in_QDA:
        print("Accuracy of class ",x," is ",(class_short_in_QDA[x]/class_short_in_real[x])*100,"%")
    print("------------------------------------------------------------------------------------")

(784, 1)


KeyboardInterrupt: 